In [1]:
import os
import numpy as np
import time
import sys

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision
import torch.optim as optim
import torch.nn.functional as tfunc
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as func
from torch.utils.data import DataLoader

from sklearn.metrics.ranking import roc_auc_score

from DatasetGenerator import DatasetGenerator
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms
from PIL import Image
import torchnet as tnt
import pandas as pd
from skimage.transform import resize
import random
import matplotlib
import cv2

import matplotlib.pyplot as plt
from skimage.io import imsave

nclasses = 3
confusion_meter = tnt.meter.ConfusionMeter(nclasses, normalized=True)
from tqdm import tqdm
from model import FCDenseNet103, FCDenseNet57, FCDenseNet67
#--------------------------------------------------------------------------------
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home/uavws/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
modelWC = FCDenseNet57(nclasses, 5)
saved_params = torch.load('../exps/models_with_coordinate/model-m-25062018-184326-tramisu_2D_FC57_loss = 3.8611210505167644_acc = 0.9975007267321571_best_acc.pth.tar')
modelWC.load_state_dict(saved_params['state_dict'])
modelWC = modelWC.eval()

modelWOC = FCDenseNet57(nclasses, 3)
saved_params = torch.load('../exps/models_with_out_coordinate/model-m-26062018-220047-tramisu_2D_FC57_loss = 3.8915366172790526_acc = 0.9971649687168962_best_acc.pth.tar')
modelWOC.load_state_dict(saved_params['state_dict'])
modelWOC = modelWOC.eval()

modelWC_cla = FCDenseNet57(nclasses, 11) # 11 channels for clahe images concat
saved_params = torch.load('../exps/modelsclaheWC11/model-m-best_acc.pth.tar')
modelWC_cla.load_state_dict(saved_params['state_dict'])
modelWC_cla = modelWC_cla.eval()

In [3]:
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
transformList = []
transformList.append(transforms.ToTensor())
transformList.append(normalize)
transformSequence=transforms.Compose(transformList)

def apply_coordinates(image):
    # image: h, h, c
    assert image.size[0] == image.size[1] 
    x, y  = image.size
    x = np.arange(x)/image.size[0]
    y = np.arange(y)/image.size[1]
    Xmat, Ymat = np.meshgrid(x, y)
    return Xmat, Ymat

def seg_loader(path):
    seg = np.array(Image.open(path).convert('L'))
    seg = np.uint8(resize(seg, (512, 512), order=0)*255)
    seg[seg <= 50] = 1
    seg[(seg > 50)*(seg <= 200)] = 2
    seg[seg > 200] = 0
    seg = seg[0:min(seg.shape), 0:min(seg.shape)]
    return np.uint8(seg)

def apply_coordinates(image):
    # image: h, h, c
    assert image.size[0] == image.size[1] 
    x, y  = image.size
    x = np.arange(x)/image.size[0]
    y = np.arange(y)/image.size[1]
    Xmat, Ymat = np.meshgrid(x, y)
    return Xmat, Ymat


def clahe_single(ori_img,clipLimit,tileGridSize):
    ori_img = np.uint8(ori_img)
    lab = cv2.cvtColor(ori_img, cv2.COLOR_RGB2LAB)
    
    lab_planes = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit,tileGridSize)
    lab_planes[0] = clahe.apply(lab_planes[0])
    
    lab = cv2.merge(lab_planes)
    rgb = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

    return rgb

def clahe_all(ori_img):
    rgb_1 = clahe_single(ori_img, 2.0 , (8,8))
    rgb_2 = clahe_single(ori_img, 2.0, (300,300))
    return Image.fromarray(rgb_1), Image.fromarray(rgb_2)

In [4]:
from scipy.ndimage.measurements import label

def getConnectedComp(img, 
                     min_extent_thresh = 0.2,
                     min_area_per = 0.001, # 0.1% of an entire area 
                     max_area_per = 0.04,  # 4 % of an entire area
                     display=False):
    min_area_thresh = min_area_per*np.prod(img.shape) 
    max_area_thresh = max_area_per*np.prod(img.shape) 
    print (img.shape, min_area_thresh, max_area_thresh)
    
    img_mask, num = label(img)

    mask = np.zeros_like(img_mask)
    for i in range(num):
        img_temp = (img_mask == i)
        
        ar, e, ra = getaspectratio(img_temp)
        print ("aspectRatio: ", ar, " extent: ",e, " radius: ", ra/2.0)

        if np.sum(img_temp) <= min_area_thresh or np.sum(img_temp) >= max_area_thresh: continue
        #Adjust these constraints         
#         if (ar < 0.6 or ar > 1.4) or e < 0.4 : continue
        
        mask += img_temp
    mask[mask > 0] = 1
    
    if display:
        plt.imshow(mask)
        plt.pause(1)
    
    return mask*img

def getaspectratio(binary_img):
    binary_img = np.uint8(binary_img)
    im2,contours,hierarchy = cv2.findContours(binary_img, 1, 2)
    cnt = contours[0]
    x,y,w,h = cv2.boundingRect(cnt)
    aspect_ratio = float(h)/w
    
    area = cv2.contourArea(cnt)
    rect_area = w*h
    extent = float(area)/rect_area
    
    equi_diameter = np.sqrt(4*area/np.pi)
    return aspect_ratio, extent, equi_diameter

def convertLabels(mask):
    # convert label segmentations
    # 1 inner, 2 outer ring, 0 background in our case
    # 128 outer, 0 inner, 255 background
    
    mask[mask == 0] = 255
    mask[mask == 2] = 128
    mask[mask == 1] = 0
    return mask

def fitAndFillCircle(mask):
    # mask is segmentation mask
    # mask before label convertion
    # 1 in inner 2 is outer
    # average cdr = 0.3
    avg_cdr = 0.3
    cdr = 0.3
    try:
        if np.sum(mask == 1) < 200:
            mask[mask == 1] = 2
    except: pass
    
    try: 
        if np.sum(mask == 2) < 800:
            mask[mask == 2] = 0
    except: pass
    
    # ==========================================
    if np.array_equal(np.unique(mask), np.array([0, 1, 2])):
        class1 = np.uint8(mask==1)
        im2, contours, hierarchy = cv2.findContours(class1, 1, 2)
        cnt = max(contours, key = cv2.contourArea)
        ellipse1 = cv2.fitEllipse(cnt)
        print (ellipse1)
        class1 = cv2.ellipse(class1, ellipse1, 1, -1)
        class1[class1 != 1] = 0
        
        class2 = np.uint8(mask==2)
        im2, contours, hierarchy = cv2.findContours(class2, 1, 2)
        cnt = max(contours, key = cv2.contourArea)
        ellipse2 = cv2.fitEllipse(cnt)
        print (ellipse2)
        class2 = cv2.ellipse(class2, ellipse2, 2, -1)
        class2[class2 != 2] = 0
        
        cdr = np.mean(np.array(ellipse1[1])) / np.mean(np.array(ellipse2[1]))
        mask = class1 + class2
        mask[mask == 3] = 1 # overlapping region
        
    elif np.array_equal(np.unique(mask), np.array([0, 1])):
        class1 = np.uint8(mask==1)
        im2, contours, hierarchy = cv2.findContours(class1, 1, 2)
        cnt =  max(contours, key = cv2.contourArea)
        ellipse1 = cv2.fitEllipse(cnt)
        print (ellipse1)
        class1 = cv2.ellipse(class1, ellipse1, 1, -1)
        class1[class1 != 1] = 0
        
        class2 = np.zeros_like(class1)
        ellipse2 = (ellipse1[0], np.array(ellipse1[1])/avg_cdr, ellipse1[2])
        print (ellipse2)
        class2 = cv2.ellipse(class2, ellipse2, 2,-1)
        class2[class2 != 2] = 0
        
        cdr = np.mean(np.array(ellipse1[1])) / np.mean(np.array(ellipse2[1]))
        mask = class1 + class2
        mask[mask == 3] = 1 # overlapping region
    
    elif np.array_equal(np.unique(mask), np.array([0, 2])):
        class2 = np.uint8(mask==2)
        im2, contours, hierarchy = cv2.findContours(class2, 1, 2)
        cnt =  max(contours, key = cv2.contourArea)
        ellipse2 = cv2.fitEllipse(cnt)
        print (ellipse2)
        class2 = cv2.ellipse(class2, ellipse2, 2, -1)
        class2[class2 != 2] = 0
        
        class1 = np.zeros_like(class2)
        ellipse1 = (ellipse2[0], np.array(ellipse2[1])*avg_cdr, ellipse2[2])
        print (ellipse1)
        class1 = cv2.ellipse(class1, ellipse1, 1, -1)
        class1[class1 != 1] = 0
        
        cdr = np.mean(np.array(ellipse1[1])) / np.mean(np.array(ellipse2[1]))
        mask = class1 + class2
        mask[mask == 3] = 1 # overlapping region
    
    else:
        class2 = np.uint8(np.zeros_like(mask))
        ellipse2 = ((304.8247985839844, 812.7166748046875), (256.9627685546875, 294.8753356933594), 85.6)
#         ellipse2 = ((1304.8247985839844, 1512.7166748046875), (256.9627685546875, 294.8753356933594), 85.6)
        print (ellipse2)
        class2 = cv2.ellipse(class2, ellipse2, 2, -1)
        class2[class2 != 2] = 0
        
        class1 = np.zeros_like(class2)
        ellipse1 = (ellipse2[0], np.array(ellipse2[1])*avg_cdr, ellipse2[2])
        print (ellipse1)
        class1 = cv2.ellipse(class1, ellipse1, 1, -1)
        class1[class1 != 1] = 0
        
        cdr = np.mean(np.array(ellipse1[1])) / np.mean(np.array(ellipse2[1]))
        mask = class1 + class2
        mask[mask == 3] = 1 # overlapping region
        
#     plt.subplot(1,3,1)
#     plt.imshow(class1)
#     plt.subplot(1,3,2)
#     plt.imshow(class2)
#     plt.subplot(1,3,3)
#     plt.imshow(mask)
#     plt.pause(1)
    del class1, class2
    
    # cdr = outer/inner radius
    # class1/class2
    return mask, cdr



def preprocessing(img):
    img = np.array(img)
    kernel = np.ones((10,10),np.float32)/100
    img = cv2.filter2D(img,-1,kernel)

    equ = cv2.equalizeHist(cv2.cvtColor(img,cv2.COLOR_RGB2GRAY))
    equ = equ>=250

    kernel2 = np.ones((100,100),np.float32)/10000   
    a = np.expand_dims(equ, axis=2)*1.0
    equ_avg = cv2.filter2D(a,-1,kernel2)

    kernel3 = np.ones((400,400),np.float32)/400**2   
    b = np.expand_dims(equ_avg, axis=2)*1.0
    dilation = cv2.dilate(b,kernel3,iterations = 1)

    dilation = dilation>=(np.max(dilation)*0.95)*1.0
    dilation = np.expand_dims(dilation, axis=2)*1.0

    new_img = np.zeros_like(img)
    new_img[:,:,0] = img[:,:,0]*dilation[:,:,0]
    new_img[:,:,1] = img[:,:,1]*dilation[:,:,0]
    new_img[:,:,2] = img[:,:,2]*dilation[:,:,0]

    return dilation[:, :, 0]

In [7]:
test_dir = '../../DATA/REFUGE-Validation400/REFUGE-Validation400'
save_dir = '../..//DATA/REFUGE-Validation400/REFUGE-Validation400_segmentations'

os.makedirs(save_dir, exist_ok=True)
test_images = os.listdir(test_dir)

In [17]:
from skimage.transform import resize

def getSegmentation(img, 
                     model, 
                     cla=True, 
                     coordinate=True):
    # img is PIL object
    model = model.eval()
    
    img = img.convert('RGB').crop((0, 0, min(img.size), min(img.size))).resize((512,512))
    image = transformSequence(img)
    
    if cla: 
        rgb1, rgb2 = clahe_all(np.array(img))
        rgb1= transformSequence(rgb1)
        rgb2= transformSequence(rgb2)
        image = torch.cat([image, rgb1, rgb2], 0)
        
    if coordinate:
        Xmat, Ymat = apply_coordinates(img)
        Xmat = torch.FloatTensor(Xmat).unsqueeze(0)
        Ymat = torch.FloatTensor(Ymat).unsqueeze(0)
        image = torch.cat([image, Xmat, Ymat], 0)
        
    seg = model(image.unsqueeze(0))
    del model
    
    return seg

def argMaxOnLogits(seg,
                   orig_img,
                   mask,
                   post_processing = {'ma'},
                   save = None):
    
    
    pred_temp = np.zeros(orig_img.size)

    _, pred = torch.max(seg, 1)
    pred = pred.data.cpu().numpy()
    pred = pred.transpose(2, 1, 0)[:,:,0].T
    
    #pred = resize(pred, (min(orig_img.size), min(orig_img.size)), order = 0)# 0 for  Nearest-neighbor
    
    pred = cv2.resize(pred, (min(orig_img.size), min(orig_img.size)), interpolation=cv2.INTER_NEAREST)
    pred_temp[0:min(orig_img.size), 0:min(orig_img.size)] = pred

    
    pred = np.uint8(pred_temp)
    
    if 'ma' in post_processing:
        pred = pred*mask
        
    if 'cc' in post_processing:
        pred = getConnectedComp(pred)
    
    if 'cf' in post_processing:
        pred, cdr = fitAndFillCircle(pred)

    
        
    pred = convertLabels(pred)
    
    if save != None:
        imsave(save, pred)
        
#     print ("final: {}".format(np.unique(pred))," CDR: {}".format(cdr))
    
    return pred

In [20]:
plt.ion()
for timage in tqdm(test_images):
    img = Image.open(os.path.join(test_dir, timage))
    mask = preprocessing(img)
    
    
    seg1 = getSegmentation(img, modelWOC, cla=False, coordinate=False)
    seg2 = getSegmentation(img, modelWC, cla=False, coordinate=True)
    seg3 = getSegmentation(img, modelWC_cla, cla=True, coordinate=True)
    
    seg_am = (seg1 + seg2 + seg3)/3.
    seg_gm = torch.pow((seg1 + 1.)*(seg2 + 1.)*(seg3 + 1.), 1./3.) - 1.
    
    pred1 = argMaxOnLogits(seg1, img, mask)
    pred2 = argMaxOnLogits(seg2, img, mask)
    pred3 = argMaxOnLogits(seg3, img, mask, save = os.path.join(save_dir, timage.split('.')[0] + '.bmp'))
    pred_am = argMaxOnLogits(seg_am, img, mask)
    pred_gm = argMaxOnLogits(seg_gm, img, mask)
    
    ### visualization............
    
    plt.subplot(2, 4, 1)
    plt.imshow(np.array(img))
    
    plt.subplot(2, 4, 2)
    plt.imshow(pred1, cmap='gray')
    
    plt.subplot(2, 4, 3)
    plt.imshow(pred2, cmap='gray')
    
    plt.subplot(2, 4, 4)
    plt.imshow(pred3, cmap='gray')
    
    plt.subplot(2, 4, 5)
    plt.imshow(pred_am, cmap='gray')
    
    plt.subplot(2, 4, 6)
    plt.imshow(img*np.expand_dims(pred2>0, 2), cmap='gray')
    
    plt.subplot(2, 4, 7)
    plt.imshow(img*np.expand_dims(pred3>0, 2), cmap='gray')
    
    plt.subplot(2, 4, 8)
    plt.imshow(img*np.expand_dims(pred_am>0, 2), cmap='gray')
    
    plt.pause(0.5)

  0%|          | 0/400 [00:00<?, ?it/s]


ValueError: Images of type float must be between -1 and 1.

In [8]:
###### odWC, ocWC, odWOC, ocWOC, odWC_cla, ocWC_cla = [], [], [], [], [], []
# plt.ion()

for timage in test_images:
    img = Image.open(os.path.join(test_dir, timage))
    img = img.convert('RGB').crop((0, 0, min(img.size), min(img.size))).resize((512,512))
    rgb1, rgb2 = clahe_all(np.array(img))
#     gt = seg_loader(os.path.join(test_dir.replace('Images', 'Masks'), timage.replace('.jpg', '.bmp')))
    
    image = transformSequence(img)
    rgb1= transformSequence(rgb1)
    rgb2= transformSequence(rgb2)
    image3 = torch.cat([image, rgb1, rgb2], 0)
    
    Xmat, Ymat = apply_coordinates(img)
    Xmat = torch.FloatTensor(Xmat).unsqueeze(0)
    Ymat = torch.FloatTensor(Ymat).unsqueeze(0)
    
#     image = torch.cat([image, Xmat, Ymat], 0)
    seg1   = modelWOC(image.unsqueeze(0))
    seg2   = modelWC(torch.cat([image, Xmat, Ymat], 0).unsqueeze(0))
    seg3   = modelWC_cla(torch.cat([image3, Xmat, Ymat], 0).unsqueeze(0)) 
    
    _, pred1 = torch.max(seg1, 1)
    pred1 = pred1.data.cpu().numpy()
    pred1 = pred1.transpose(2, 1, 0)[:,:,0].T
    pred1 = getConnectedComp(pred1)
    pred1,_ = fitAndFillCircle(pred1)
#     print (np.unique(pred1))
    
    _, pred2 = torch.max(seg2, 1)
    pred2 = pred2.data.cpu().numpy()
    pred2 = pred2.transpose(2, 1, 0)[:,:,0].T
    pred2 = getConnectedComp(pred2)
    pred2, _ = fitAndFillCircle(pred2)
    
    _, pred3 = torch.max(seg3, 1)
    pred3 = pred3.data.cpu().numpy()
    pred3 = pred3.transpose(2, 1, 0)[:,:,0].T
    pred3 = getConnectedComp(pred3)
    pred3, _ = fitAndFillCircle(pred3)
    
    seg_am    = (seg1 + seg2 + seg3)/3.0
    _, pred_am = torch.max(seg_am, 1)
    pred_am = pred_am.data.cpu().numpy()
    pred_am = pred_am.transpose(2, 1, 0)[:,:,0].T
    pred_am = getConnectedComp(pred_am)
    pred_am, _ = fitAndFillCircle(pred_am)
    
    seg_gm    = torch.pow((seg1+1.)*(seg2+1.)*(seg3+1.), 1.0/3.) -1.
    _, pred_gm = torch.max(seg_gm, 1)
    pred_gm = pred_gm.data.cpu().numpy()
    pred_gm = pred_gm.transpose(2, 1, 0)[:,:,0].T
    pred_gm = getConnectedComp(pred_gm)
    pred_gm, _ = fitAndFillCircle(pred_gm)
    
    
    
    ### visualization............
    
    plt.subplot(3, 4, 1)
    plt.imshow(np.array(img))
    
    plt.subplot(3, 4, 2)
    plt.imshow(pred1, cmap='gray')
#     plt.xlabel('3 OD {:.2f}'.format((np.sum(2.*(pred1==1)*(gt==1))/(np.sum(pred1==1)+np.sum(gt==1)))) + ' OC {:.2f}'.format((np.sum(2.*(pred1==2)*(gt==2))/(np.sum(pred1==2)+np.sum(gt==2)))))
    
    plt.subplot(3, 4, 3)
    plt.imshow(pred2, cmap='gray')
#     plt.xlabel('5 OD {:.2f}'.format((np.sum(2.*(pred2==1)*(gt==1))/(np.sum(pred2==1)+np.sum(gt==1)))) + ' OC {:.2f}'.format((np.sum(2.*(pred2==2)*(gt==2))/(np.sum(pred2==2)+np.sum(gt==2)))))
    
    plt.subplot(3, 4, 4)
    plt.imshow(pred3, cmap='gray')
#     plt.xlabel('11 OD {:.2f}'.format((np.sum(2.*(pred3==1)*(gt==1))/(np.sum(pred3==1)+np.sum(gt==1)))) + ' OC {:.2f}'.format((np.sum(2.*(pred3==2)*(gt==2))/(np.sum(pred3==2)+np.sum(gt==2)))))

    plt.subplot(3, 4, 5)
    plt.imshow(np.array(img))
    
    plt.subplot(3, 4, 6)
    plt.imshow(getConnectedComp(pred1), cmap='gray')
#     plt.xlabel('3 OD {:.2f}'.format((np.sum(2.*(pred1==1)*(gt==1))/(np.sum(pred1==1)+np.sum(gt==1)))) + ' OC {:.2f}'.format((np.sum(2.*(pred1==2)*(gt==2))/(np.sum(pred1==2)+np.sum(gt==2)))))
    
    plt.subplot(3, 4, 7)
    plt.imshow(getConnectedComp(pred2), cmap='gray')
#     plt.xlabel('5 OD {:.2f}'.format((np.sum(2.*(pred2==1)*(gt==1))/(np.sum(pred2==1)+np.sum(gt==1)))) + ' OC {:.2f}'.format((np.sum(2.*(pred2==2)*(gt==2))/(np.sum(pred2==2)+np.sum(gt==2)))))
    
    plt.subplot(3, 4, 8)
    plt.imshow(getConnectedComp(pred3), cmap='gray')



    plt.subplot(3, 4, 9)
    plt.imshow(np.array(img))
    
    plt.subplot(3, 4, 10)
    plt.imshow(np.array(img))
    
    plt.subplot(3, 4, 11)
    plt.imshow(pred_am, cmap='gray')
    plt.xlabel(str(np.unique(pred_am)))
    
    plt.subplot(3, 4, 12)
    plt.imshow(pred_gm, cmap='gray')
    plt.xlabel(str(np.unique(pred_gm)))


#     imsave('/home/dlguru/Varghese/Refuge/data/Testing/Disc_Cup_Seg_without_coordinates/'+ timage, 120*pred.transpose(2, 1, 0)[:,:,0].T)
    plt.pause(5)
    
#     print ("====================================")
#     print ('Optic Disk: ', (np.sum(2.*(pred1==1)*(gt==1))/(np.sum(pred1==1)+np.sum(gt==1))), ' Optic cup: ', (np.sum(2.*(pred1==2)*(gt==2))/(np.sum(pred1==2)+np.sum(gt==2))))
#     print ('Optic Disk: ', (np.sum(2.*(pred2==1)*(gt==1))/(np.sum(pred2==1)+np.sum(gt==1))), ' Optic cup: ', (np.sum(2.*(pred2==2)*(gt==2))/(np.sum(pred2==2)+np.sum(gt==2))))
#     print ('Optic Disk: ', (np.sum(2.*(pred3==1)*(gt==1))/(np.sum(pred3==1)+np.sum(gt==1))), ' Optic cup: ', (np.sum(2.*(pred3==2)*(gt==2))/(np.sum(pred3==2)+np.sum(gt==2))))
    
#     odWOC.append((np.sum(2.*(pred1==1)*(gt==1))/(np.sum(pred1==1)+np.sum(gt==1))))
#     ocWOC.append((np.sum(2.*(pred1==2)*(gt==2))/(np.sum(pred1==2)+np.sum(gt==2))))
    
#     odWC.append((np.sum(2.*(pred2==1)*(gt==1))/(np.sum(pred2==1)+np.sum(gt==1))))
#     ocWC.append((np.sum(2.*(pred2==2)*(gt==2))/(np.sum(pred2==2)+np.sum(gt==2))))
    
#     odWC_cla.append((np.sum(2.*(pred3==1)*(gt==1))/(np.sum(pred3==1)+np.sum(gt==1))))
#     ocWC_cla.append((np.sum(2.*(pred3==2)*(gt==2))/(np.sum(pred3==2)+np.sum(gt==2))))
    

(512, 512) 262.144 10485.76
aspectRatio:  1.0  extent:  0.2222222222222222  radius:  0.7978845608028654
aspectRatio:  1.0  extent:  0.2222222222222222  radius:  0.7978845608028654
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.5  extent:  0.16666666666666666  radius:  0.5641895835477563
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.2222222222222222  radius:  0.7978845608028654
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.2222222222222222  radius:  0.7978845608028654
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  0.5  extent:  0.0  radius

(512, 512) 262.144 10485.76
aspectRatio:  1.5  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.2222222222222222  radius:  0.7978845608028654
aspectRatio:  3.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  0.5  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  ext

/home/brats/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


(512, 512) 262.144 10485.76
(512, 512) 262.144 10485.76
(512, 512) 262.144 10485.76
aspectRatio:  1.1159420289855073  extent:  0.7533408620365142  radius:  35.693631357016926
(512, 512) 262.144 10485.76
aspectRatio:  1.0  extent:  0.2222222222222222  radius:  0.7978845608028654
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.2222222222222222  radius:  0.7978845608028654
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.8571428571428572  extent:  0.3516483516483517  radius:  3.1915382432114616
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.2222222222222222  radius:  0.7978845608028654
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.2222222222222222  radius:  0.79788

((129.92266845703125, 264.0558166503906), (47.06727981567383, 61.50063705444336), 1.5878628492355347)
((129.92266845703125, 264.0558166503906), array([14.12018394, 18.45019112]), 1.5878628492355347)
(512, 512) 262.144 10485.76
aspectRatio:  1.5  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.2222222222222222  radius:  0.7978845608028654
aspectRatio:  2.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectRatio:  1.0  extent:  0.0  radius:  0.0
aspectR

KeyboardInterrupt: 

In [203]:
timage = test_images[0]
gt = seg_loader(os.path.join(test_dir.replace('Images', 'Masks'), timage.replace('.jpg', '.bmp')))
class1 = np.uint8(gt==1)
plt.imshow(class1)
plt.xlabel("class1")
plt.pause(1)
im2, contours, hierarchy = cv2.findContours(class1, 1, 2)
cnt = max(contours, key = cv2.contourArea)
ellipse = cv2.fitEllipse(cnt)
print (ellipse)
cv2.ellipse(class1,ellipse,1,-1)

class2 = np.uint8(gt==2)
plt.imshow(class2)
plt.xlabel("class2")
plt.pause(1)
im2, contours, hierarchy = cv2.findContours(class2, 1, 2)
cnt = max(contours, key = cv2.contourArea)

ellipse = cv2.fitEllipse(cnt)
print (ellipse)
class2 = cv2.ellipse(class2,ellipse, 255, -1)
plt.imshow(class2)
plt.xlabel("fill ellipse")
plt.pause(1)

mask = class1 + class2
mask[mask == 3] = 1 # 
plt.imshow(mask)
plt.xlabel("gt")
plt.show()

/home/brats/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


((96.65797424316406, 221.0191650390625), (20.629261016845703, 21.411630630493164), 54.32870101928711)
((97.54911041259766, 220.7281036376953), (45.563480377197266, 50.55513000488281), 137.1709442138672)


In [ ]:
df = pd.DataFrame()
df['OpticDiskWC'] = odWC
df['OpticCupWC'] = ocWC

df['OpticDiskWOC'] = odWOC
df['OpticCupWOC'] = ocWOC

df['OpticDiskWC_CLA'] = odWC_cla
df['OpticCupWC_CLA'] = ocWC_cla
df.describe()
#df.to_csv('./output.csv')
# without coordinates.....

In [ ]:
df = pd.DataFrame()
df['OpticDisk'] = od
df['OpticCup'] = oc
df.describe()